In [179]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import sklearn as sk
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn.ensemble

from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

import pickle
import copy
import random
import sys

def open_pickle(name):
    with open(name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        x, y = u.load()
        return (x, y)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [213]:
class ClassificationTree:
    class Vertex:
        def __init__(self, f=None, div=None, l=None, ge=None, res=None):
            self.f = f
            self.div = div
            self.l = l
            self.ge = ge
            self.res = res
        
    def __init__(self, max_h, max_v, max_search):
        self.max_h = max_h
        self.max_v = max_v
        self.max_search = max_search
        
    def _gini(self, Y, items):
        c = {}
        for i in items:
            if Y[i] in c:
                c[Y[i]] += 1
            else:
                c[Y[i]] = 1
        res = 1
        for v in c:
            res = res - (c[v] / len(items)) ** 2
        return res
    
    def _make_list(self, Y, items):
        c = {}
        for i in items:
            if Y[i] in c:
                c[Y[i]] += 1
            else:
                c[Y[i]] = 1
        mx_value = 0
        for key in c:
            if c[key] > mx_value:
                mx_value = c[key]
                mx_key = key
        return self.Vertex(res=mx_key)
    
    def _make_vertex(self, X, Y, items, features, h):
        eq = True
        for i in items:
            if Y[items[0]] != Y[i]:
                eq = False
        if eq:
            return self.Vertex(res=Y[items[0]])
        
        if h == self.max_h or len(items) <= self.max_v:
            return self._make_list(Y, items)
        
        cur_gini = self._gini(Y, items)
        #print(cur_gini, len(items))
        mn_gini = 1
        it = 0
        while it < self.max_search:
            it += 1
            f = random.choice(features)
            div = random.choice(items)
            l = []
            ge = []
            for i in items:
                if X[i][f] < X[div][f]:
                    l.append(i)
                else:
                    ge.append(i)
            if len(l) == 0 or len(ge) == 0:
                continue
            gini = self._gini(Y, l) * len(l) / len(items) + self._gini(Y, ge) * len(ge) / len(items)
            if gini < mn_gini:
                mn_gini = gini
                mn_l = l
                mn_ge = ge
                mn_div = X[div][f]
                mn_f = f
        #print(cur_gini, mn_gini)
        if cur_gini - mn_gini < 0.00001:
            return self._make_list(Y, items)
        return self.Vertex(f=mn_f, div=mn_div, l=self._make_vertex(X, Y, mn_l, features, h + 1),
                      ge=self._make_vertex(X, Y, mn_ge, features, h + 1))
                    
    def fit(self, X, Y, features=None):
        if X.shape[0] != Y.shape[0]:
            raise ValueError("Wrong X and Y shapes: " + str(X.shape) + ", " + str(Y.shape))
        if features is None:
            features = [i for i in range(X.shape[1])]
        self.root = self._make_vertex(X, Y, [i for i in range(Y.shape[0])], features, 1)
            
          
    def predict(self, X):
        res = []
        for x in X:
            cur = self.root
            while cur.res == None:
                if x[cur.f] < cur.div:
                    cur = cur.l
                else:
                    cur = cur.ge
            res.append(cur.res)
        return np.array(res)
    
class RandomForestClassifier:
    def __init__(self, features, max_search, trees, max_h=10, max_v=1):
        self.features = features
        self.max_search = max_search
        self.trees = []
        for i in range(trees):
            self.trees.append(ClassificationTree(max_h, max_v, max_search))
    
    def fit(self, X, Y):
        for i in range(len(self.trees)):
            subX = []
            subY = []
            for _ in Y:
                n = random.randint(0, len(Y) - 1)
                subX.append(X[n])
                subY.append(Y[n])
            self.trees[i].fit(np.array(subX), np.array(subY), 
                              features=np.random.permutation(X.shape[1])[0:self.features])
            
    def predict(self, X):
        y = []
        for i in range(len(self.trees)):
            y.append(self.trees[i].predict(X))
        res = []
        for i in range(len(y[0])):
            c = {}
            for j in range(len(self.trees)):
                if y[j][i] in c:
                    c[y[j][i]] += 1
                else:
                    c[y[j][i]] = 1
            mx_value = 0
            for key in c:
                if mx_value < c[key]:
                    mx_value = c[key]
                    mx_key = key
            res.append(mx_key)
        return np.array(res)
    
    

In [104]:
def cv(x, y, classificator, n_folds=10, k_iter=4):
    res = 0
    for state in range(k_iter):
        ires = 0
        kf = KFold(n=len(y), n_folds=n_folds, shuffle=True, random_state=state + 10)
        for train_index, test_index in kf:
            x_train, x_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            cl = copy.copy(classificator)
            cl.fit(x_train, y_train)
            y_pred = cl.predict(x_test)
            #print(y_pred)
            #print(y_test)
            #print("------------------")
            ires += accuracy_score(y_test, y_pred)
        res += ires / n_folds
    res = res / k_iter
    return res

In [216]:
x, y = open_pickle('iris.txt')
for h in [30, 40, 50, 60, 70, 80, 90]:
    for v in [1,4]:
        for f in [20, 30, 100]:
            ct = ClassificationTree(h, v, f)
            print(cv(x, y, ct, 10, 5), h, v, f)

0.94 30 1 20
0.945333333333 30 1 30
0.946666666667 30 1 100
0.945333333333 30 4 20
0.938666666667 30 4 30
0.937333333333 30 4 100
0.944 40 1 20
0.946666666667 40 1 30
0.950666666667 40 1 100
0.937333333333 40 4 20
0.941333333333 40 4 30
0.945333333333 40 4 100
0.944 50 1 20
0.956 50 1 30
0.948 50 1 100
0.930666666667 50 4 20
0.938666666667 50 4 30
0.94 50 4 100
0.938666666667 60 1 20
0.942666666667 60 1 30
0.949333333333 60 1 100
0.945333333333 60 4 20
0.94 60 4 30
0.942666666667 60 4 100
0.944 70 1 20
0.941333333333 70 1 30
0.953333333333 70 1 100
0.942666666667 70 4 20
0.945333333333 70 4 30
0.942666666667 70 4 100
0.937333333333 80 1 20
0.945333333333 80 1 30
0.953333333333 80 1 100
0.952 80 4 20
0.941333333333 80 4 30
0.941333333333 80 4 100
0.948 90 1 20
0.946666666667 90 1 30
0.948 90 1 100
0.944 90 4 20
0.94 90 4 30
0.944 90 4 100


In [235]:
x, y = open_pickle('iris.txt')
bests = []
for f in range(1, 5):
    for max_search in range(1, 51, 5):
        for trees in range(2, 10):
            rf = RandomForestClassifier(f, max_search, trees)
            t = cv(x, y, rf, 10, 3)
            print(t, f, max_search, trees)
            bests.append([t, f, max_search, trees])
bests = sorted(bests, key=lambda x: -x[0])
print("top:")
print(bests[:10])

0.648888888889 1 1 2
0.733333333333 1 1 3
0.76 1 1 4
0.804444444444 1 1 5
0.786666666667 1 1 6
0.842222222222 1 1 7
0.837777777778 1 1 8
0.851111111111 1 1 9
0.78 1 6 2
0.855555555556 1 6 3
0.833333333333 1 6 4
0.831111111111 1 6 5
0.848888888889 1 6 6
0.862222222222 1 6 7
0.888888888889 1 6 8
0.877777777778 1 6 9
0.742222222222 1 11 2
0.822222222222 1 11 3
0.797777777778 1 11 4
0.86 1 11 5
0.855555555556 1 11 6
0.88 1 11 7
0.871111111111 1 11 8
0.891111111111 1 11 9
0.713333333333 1 16 2
0.775555555556 1 16 3
0.84 1 16 4
0.822222222222 1 16 5
0.864444444444 1 16 6
0.88 1 16 7
0.851111111111 1 16 8
0.868888888889 1 16 9
0.793333333333 1 21 2
0.806666666667 1 21 3
0.824444444444 1 21 4
0.842222222222 1 21 5
0.806666666667 1 21 6
0.846666666667 1 21 7
0.88 1 21 8
0.897777777778 1 21 9
0.715555555556 1 26 2
0.817777777778 1 26 3
0.82 1 26 4
0.873333333333 1 26 5
0.851111111111 1 26 6
0.862222222222 1 26 7
0.866666666667 1 26 8
0.873333333333 1 26 9
0.722222222222 1 31 2
0.788888888889 1 3

In [231]:
x, y = open_pickle('iris.txt')
print(cv(x, y, sklearn.ensemble.RandomForestClassifier(), 10, 5))

    

0.952


RF работает на ирисах не лучше, чем обычный CART. Даже реализация из sklearn не сильно лучше.

Кроме того KNN показывает лучшие результаты (0.97 на реализации из sklearn и нормировкой признаков) против 0.95 у RF.

Я думаю, алгоритмы, основанные на решающих деревьях, показывает не самые лучшие результаты из-за того, что у ирисов всего 4 фичи. 

Что касается времени работы, если сравнивать KNN и RF, то нельзя сказать, что быстрее - все зависит от реализации. Но RF не нужно хранить всю обучающую выборку и обучать/предсказывать можно в несколько потоков, т.к. деревья не зависят друг от друга.

Вывод: на ирисах сложно показать всю мощью Random Forest. 

